In [1]:
import sys
sys.path.append("C:\\Users\\lauth\\OneDrive\\Desktop\\sql_assistant_v3")
from src.components.models.llms.llms import Langchain_OpenAI_LLM

llm = Langchain_OpenAI_LLM()
llm.init_model()

In [2]:
# Generando el request 

from src.app.pipeline_processes.query_pre_process.generation import generate_request
from src.components.memory.memory import Memory

memory = Memory()
user_input = "Hola como te llamas?"
actual_answer="Hello, my name M-Assistant. How can I help you?"
user_input = "I want information from EMED-2012-LR-P12"

memory.add_user_message(user_input)
memory.add_ai_message(actual_answer)

res = generate_request(llm, memory)

res


In [3]:
# Clasificando el request

from src.app.pipeline_processes.query_pre_process.generation import generate_request_type

request = "The human is requesting information from the EMED-2012-LR-P12 measurement system."
request_type = generate_request_type(llm, request)

request_type


In [9]:
# Cuando el request es SIMPLE

from src.app.pipeline_processes.query_pre_process.generation import generate_greeting_response_call
from src.components.memory.memory import Memory

memory = Memory()
user_input = "Hola me llamo lauther"

memory.add_user_message(user_input)
res = generate_greeting_response_call(llm, memory)


In [ ]:
# Cuando el request es COMPLEJO ----- Parte 1 (pre-process): Buscando tablas y columnas (semantic info)

from src.app.pipeline_processes.sql_pre_process.generation import generate_semantic_info
from src.components.models.embeddings.embeddings import Langchain_OpenAI_Embeddings

embeddings=Langchain_OpenAI_Embeddings()
embeddings.init_model()

user_request = "The human is asking for a list of measurement systems similar to EMED"

semantic_info = generate_semantic_info(llm, embeddings, user_request)
print(semantic_info.keys())

In [10]:
# Cuando el request es COMPLEJO ----- Parte 2 : Generando el SQL

from src.app.pipeline_processes.sql_generation_process.generation import generate_sql

user_request = "The human is asking for a list of measurement systems similar to EMED"
sql_code = generate_sql(llm, user_request, semantic_info)

In [2]:
sql_code_completo = "SELECT Nombre AS 'Measurement System Name', Tag AS 'Measurement System Tag' FROM dbo_v2.med_sistema_medicion WHERE Tag LIKE 'EMED%'"
sql_code_incompleto = "SELECT Nombre AS Measurement_System_Name, Tag AS Measurement_System_Tag FROM dbo_v2.med_sistema_medicion WHERE Tag = <Specific Tag>;"

semantic_info = {
    "med_sistema_medicion": [
        (
            "Id",
            "INT",
            "Unique identifier for the measurement system",
            1,
            ("fcs_computador_medidor", "med_sistema_medicion"),
        ),
        (
            "Id",
            "INT",
            "Unique identifier for the measurement system",
            1,
            "med_sistema_medicion",
        ),
        (
            "Nombre",
            "VARCHAR(100)",
            "Name of the measurement system",
            2,
            "med_sistema_medicion",
        ),
        (
            "Tag",
            "VARCHAR(100)",
            "Tag associated to the measurement system",
            3,
            "med_sistema_medicion",
        ),
        (
            "Estado",
            "VARCHAR(100)",
            "Status of the measurement system, active ('ACT') or inactive ('INA')",
            4,
            "med_sistema_medicion",
        ),
        (
            "Uso",
            "VARCHAR(100)",
            "Usage description of the measurement system could be: CONTINUA / WELL_TESTING",
            6,
            "med_sistema_medicion",
        ),
        (
            "IdPlataforma_fk",
            "INT",
            "Foreign key linking measurement systems with platforms",
            999,
            ("med_sistema_medicion", "pla_plataforma"),
        ),
        (
            "IdAplicabilidad_fk",
            "INT",
            "Foreign key linking measurement system with their measure type",
            999,
            ("med_tipo_medicion", "med_sistema_medicion"),
        ),
    ],
    "med_tipo_medicion": [
        (
            "Id",
            "INT",
            "Unique identifier for each measure type",
            1,
            "med_tipo_medicion",
        ),
        (
            "Id",
            "INT",
            "Unique identifier for each measure type",
            1,
            ("med_tipo_medicion", "med_sistema_medicion"),
        ),
        (
            "Nombre",
            "VARCHAR(150)",
            "Name of the measurement type",
            2,
            "med_tipo_medicion",
        ),
        (
            "Estado",
            "VARCHAR(10)",
            "Status of the measurement type, active ('ACT') or inactive ('INA')",
            3,
            "med_tipo_medicion",
        ),
    ]
}

##

In [3]:
# Cuando el request es COMPLEJO ----- Parte 3 (post-process): Clasificacion de SQL Generado (¿Esta completo o tiene placeholders?)

from src.app.pipeline_processes.sql_post_process.generation import generate_sql_classification

sql_class = generate_sql_classification(llm, sql_code_incompleto, semantic_info)
sql_class

{'analysis': 'This SQL query is incomplete because it contains a placeholder that needs to be replaced by the user.',
 'class': 'incomplete',
 'suggestion': 'The user needs to provide a specific tag to replace the placeholder in order to have a complete and correct SQL query.'}

In [4]:
# Cuando el request es COMPLEJO ----- Parte 4 (post-process): Generando un Pre query si es necesario (¿Esta completo o tiene placeholders?)

from src.app.pipeline_processes.sql_post_process.generation import generate_sql_pre_query

assistant_analysis="This SQL query is incomplete because it contains a placeholder that needs to be replaced by the user."
assistant_suggestion="The user needs to provide a specific tag to replace the placeholder in order to have a complete and correct SQL query."

sql_pre_query = generate_sql_pre_query(llm, sql_code_incompleto, assistant_analysis, assistant_suggestion)
sql_pre_query

{'analysis': 'The user is requesting a pre query that will help them find the missing attributes in the incomplete SQL query.',
 'sql_pre_query': 'SELECT DISTINCT Tag FROM dbo_v2.med_sistema_medicion;',
 'tables': 'dbo_v2.med_sistema_medicion',
 'intention': 'The intention of this pre query is to retrieve a list of distinct tags from the dbo_v2.med_sistema_medicion table, which the user can then use to replace the placeholder in the incomplete SQL query and obtain the missing attributes.'}

In [7]:
# Cuando el request es COMPLEJO ----- Parte 4: Generando la respuesta al resultado sql.

from src.utils.sql_utils import run_sql
from src.app.pipeline_processes.sql_post_process.generation import (
    generate_sql_summary_response,
)

completo_user_request = "The human is asking for a list of measurement systems similar to EMED"
# incompleto_user_request = "The user is asking for a specific measurement system tag"
is_prequery = False

res_dataframe = run_sql(sql_code_completo)

sql_summary = generate_sql_summary_response(
    llm,
    res_dataframe,
    completo_user_request,
    is_prequery
)
sql_summary

C:\Users\lauth\OneDrive\Desktop\sql_assistant_v3\src\utils\sql_utils.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(sql, conn)


{'user_request': 'The human is asking for a list of measurement systems similar to EMED',
 'response': 'Here is a list of measurement systems similar to EMED: EMED-3138.11-128, EMED-3138.12-050, EMED-3138.12-052, EMED-3138.12-065, EMED-3138.12-105, EMED-3138.12-015, EMED-3138.12-102, EMED-3135.01-067, EMED-3161.01-060, EMED-3161.01-061. Please select an item from the list for more information.'}

In [9]:
print(llm.responses[1].json_response)

{'user_request': 'The human is asking for a list of measurement systems similar to EMED', 'response': 'Here is a list of measurement systems similar to EMED: EMED-3138.11-128, EMED-3138.12-050, EMED-3138.12-052, EMED-3138.12-065, EMED-3138.12-105, EMED-3138.12-015, EMED-3138.12-102, EMED-3135.01-067, EMED-3161.01-060, EMED-3161.01-061. Please select an item from the list for more information.'}


In [4]:
# Procesando la respuesta final

from app.pipeline_processes.query_post_process.generation import translate_response

user_input = "Hola como te llamas?"
actual_answer="Hello, my name M-Assistant. How can I help you?"

res = translate_response(llm, user_input, actual_answer)

In [5]:
res

{'detected_language': 'spanish',
 'response': '¡Hola, mi nombre es M-Assistant. ¿En qué puedo ayudarte?'}

In [6]:
print(llm.get_responses())

[Base_LLM_Response(type=generate-request, prompt=The following is a conversation between ..., llm_response=
intention: The human is requesting information from the EMED-2012-LR-P12 measurement system., json_response={'intention': 'The human is requesting information from the EMED-2012-LR-P12 measurement system.'}), Base_LLM_Response(type=post-process-translation, prompt=Your task is to translate Text_2, follow..., llm_response=
detected_language: spanish
response: ¡Hola, mi nombre es M-Assistant. ¿En qué puedo ayudarte?, json_response={'detected_language': 'spanish', 'response': '¡Hola, mi nombre es M-Assistant. ¿En qué puedo ayudarte?'})]


In [14]:
print(llm.responses[1].json_response)

{'detected_language': 'spanish', 'response': '¡Hola, mi nombre es M-Assistant. ¿En qué puedo ayudarte?'}
